In [92]:
# !pip install yargy

In [93]:
# !pip install ipymarkup

In [94]:
import gzip
from yargy import Parser, rule, and_, not_, or_
from yargy.predicates import eq, in_, type, normalized, dictionary, gte, lte
from yargy.interpretation import fact
from yargy.predicates import gram, is_capitalized
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

In [95]:
from dataclasses import dataclass
from typing import Iterator

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("../Data/news.txt.gz"))

In [96]:
def test(rule, *tests):
    parser = Parser(rule)
    for line, etalon in tests:
        match = parser.match(line)
        assert match
        assert line
        guess = match.fact
        assert etalon == guess, guess

In [97]:
from ipymarkup import show_span_ascii_markup as show_markup

def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
     

In [98]:
from random import seed, sample

seed(3)
for text in sample(texts, 3):
    print(text)
    print('---' * 10)

Text(label='media', title='Вещатели назвали переоцененным влияние Солнца на телесигнал', text='В ФГУП «Российская телевизионная радиовещательная сеть» (РТРС) опровергли информацию о том, что 8 марта в ряде городов страны вещание прервется из-за повышенной солнечной активности. Об этом говорится в комментарии компании, поступившем в редакцию «Ленты.ру».Таким образом в РТРС прокомментировали появившуюся в СМИ информацию о том, что из-за солнечной интерференции (явление, когда Солнце, спутник связи и приемная антенна наземного ретранслятора выстраиваются на одной линии) на экранах телевизоров в Международный женский день могут возникнуть значительные помехи, а где-то сигнал может и вовсе пропасть.«Солнце — мощный источник радиосигнала — оказавшись ровно позади спутника связи, также "передает" свой сигнал наземному ретранслятору. Этот сигнал способен вызвать кратковременные — не более пяти минут — ухудшения качества приема в отдельных районах страны. На телевизионных экранах могут появлять

In [99]:

Name = fact(
    'Name',
    ['first', 'last'],
)

gnc = gnc_relation()

LAST = and_(
    gram('Surn'),
    not_(gram('Abbr')),
    is_capitalized(),
).interpretation(
    Name.last.inflected()
).match(gnc)

FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
    is_capitalized(),
).interpretation(
    Name.first.inflected()
).match(gnc)


NAME = or_(
    rule(
        FIRST,
        LAST
    ), 
    rule(
        LAST,
        FIRST
    ),
        rule(
        LAST,
    ),
    rule(
        FIRST,
    ),
).interpretation(
    Name
)

print("Name")
show_matches(
    NAME,
    'управляющий директор Иван Ульянов',
    'Иванов Андрей играет на банжо',
    'Виктор Андреевич Сухоруков развивает навыки скоростного чтения',
    'Сергей отдыхает в Анапе',

)

Name
управляющий директор Иван Ульянов
                     ────────────
Иванов Андрей играет на банжо
─────────────                
Виктор Андреевич Сухоруков развивает навыки скоростного чтения
──────           ─────────                                    
Сергей отдыхает в Анапе
──────                 


In [100]:
Entry = fact(
    'Entry',
    ['name', 'birth_date', 'birth_place']
)

Date = fact(
    'Date',
    ['year', 'month', 'day']
)

gnc = gnc_relation()


Birth_place = fact(
    'Birth_place',
    ['birth_place']
)


MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

YEAR = and_(
    gte(1700),
    lte(2100)
).interpretation(
    Date.year.custom(int)
)

DATE = or_(
    rule(
        DAY,
        '.',
        MONTH_NAME,
        '.',
        YEAR
    ),
    rule(
        gram("PREP").optional(),
        DAY.optional(),
        MONTH_NAME.optional(),
        YEAR,
        normalized('год').optional()
    )
).interpretation(
    Date
)

PLACE_TAG = morph_pipeline([
    "город", 
    "село",
    "поселок",
    "деревня",
])


BIRTH_PLACE = rule(
    gram('PREP'),
    PLACE_TAG.optional(),
    and_(
        is_capitalized(),
        gram("NOUN")
    ).interpretation(Birth_place.birth_place.inflected())
).interpretation(Birth_place)

print("Date")
show_matches(
    DATE,
    'родился в городе Москва 21 февраля 1990 года',
    'родиться 32 сентября 2000 ',
    'родилась 01 июль 1917 года',
    '12 декабря 2012',
)

print("Birth place")
show_matches(
    BIRTH_PLACE,
    'родился в городе Москва 21 февраля 1990 года ',
    'родиться 32 сентября 2000 в семье ученых',
    'родилась 01 июль 1917 года в Шабаново',
    '12 декабря 2012 в Томске',
)

Date
родился в городе Москва 21 февраля 1990 года
                        ────────────────────
родиться 32 сентября 2000 
            ───────────── 
родилась 01 июль 1917 года
         ─────────────────
12 декабря 2012
───────────────
Birth place
родился в городе Москва 21 февраля 1990 года 
        ───────────────                      
родиться 32 сентября 2000 в семье ученых
родилась 01 июль 1917 года в Шабаново
                           ──────────
12 декабря 2012 в Томске
                ────────


In [101]:
# ENTRY = rule(
#     NAME.interpretation(
#         Entry.name
#     ).match(gnc),
#     BIRTH.interpretation(
#         Entry.birth
#     )
# ).interpretation(
#     Entry
# )

ENTRY = or_(
    rule(
        NAME.interpretation(Entry.name),
        normalized('родиться').optional(),
        DATE.interpretation(Entry.birth_date),
        BIRTH_PLACE.interpretation(Entry.birth_place),    
    ),
    rule(
        NAME.interpretation(Entry.name),
        normalized('родиться').optional(),
        BIRTH_PLACE.interpretation(Entry.birth_place),
        DATE.interpretation(Entry.birth_date),
    ),
).interpretation(Entry)

parser = Parser(ENTRY)

print("Name")
show_matches(
    NAME,
    'Иван Ульянов родился 30 ноября 1960 года в Саратовe',
)

print("Date")
show_matches(
    DATE,
    'Иван Ульянов родился 25 ноября 1960 года в Саратовe',
)

print("Birth Place")
show_matches(
    BIRTH_PLACE,
    'Иван Ульянов родился 30 ноября 1960 года в городе Саратов',
)

print("Entry")
show_matches(
    ENTRY,
    'Писатель Иван Ульянов родился в Саратове 25 ноября 1960 года в семье ученых',
)

match = parser.match('Иван Ульянов родился 30 ноября 1960 года в городе Саратов')
print(match)


Name
Иван Ульянов родился 30 ноября 1960 года в Саратовe
────────────                                       
Date
Иван Ульянов родился 25 ноября 1960 года в Саратовe
                     ───────────────────           
Birth Place
Иван Ульянов родился 30 ноября 1960 года в городе Саратов
                                         ────────────────
Entry
Писатель Иван Ульянов родился в Саратове 25 ноября 1960 года в семье 
         ───────────────────────────────────────────────────         
ученых
Match(tokens=[MorphToken(value='Иван', span=[0, 4), type='RU', forms=[Form('иван', Grams(NOUN,Name,anim,masc,nomn,sing))]), MorphToken(value='Ульянов', span=[5, 12), type='RU', forms=[Form('ульянов', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))]), MorphToken(value='родился', span=[13, 20), type='RU', forms=[Form('родиться', Grams(VERB,indc,intr,masc,past,perf,sing)), Form('родиться', Grams(VERB,impf,indc,intr,masc,past,sing))]), Token(value='30', span=[21, 23), type='INT'), MorphToken(value='ноября

In [119]:
show_matches(
    ENTRY,
    'Джон родился в Польше 11 сентября 1865 года , был самым младшим из семи детей в семье',
)

match = parser.match('В Театре имени Вахтангова состоялось прощание с народным артистом СССР Юрием Яковлевым. Как сообщает ИТАР-ТАСС, церемония прошла без гражданской панихиды - традиционные речи было решено заменить траурной музыкой и записью стихов, прочитанных самим актером.Одним из первых с артистом простился художественный руководитель театра Римас Туминас. На церемонии прощания также присутствовали Людмила Максакова, Марианна Вертинская, Андрей Мягков, Василий Лановой, Вера Васильева, Сергей Юрский, Эммануил Виторган, Альберт Филозов и другие артисты. В конце прощания вся труппа Вахтанговского театра поднялась на сцену. Юрия Яковлева похоронят на Новодевичьем кладбище Москвы.Юрий Яковлев скончался 30 ноября на 86-м году жизни в больнице, куда попал после обморока. Его памяти была посвящена церемония вручения премии «Звезда театрала», которая проходила 2 декабря в Театре Вахтангова.Материалы по теме18:50 30 ноября 2013Последняя пристаньПамяти Юрия ЯковлеваМатериалы по теме17:29 30 ноября 2013Глаза князя МышкинаУмер Юрий ЯковлевЮрий Яковлев родился в Москве в 1928 году. После окончания Щукинского училища в 1952 году он поступил в труппу Театра имени Вахтангова, в котором прослужил более 60 лет. За эти годы он сыграл в нем около 40 ролей. Свою последнюю роль он исполнил в спектакле «Пристань», который в 2011 году поставил Римас Туминас.Яковлев получил широкую известность и как киноактер. За свою карьеру он сыграл более 70 ролей в кино; в число самых известных его работ входят роли в фильмах «Идиот», «Иван Васильевич меняет профессию», «Гусарская баллада», «Ирония судьбы, или с легким паром» и «Кин-дза-дза». Последний раз в кино Яковлев снялся в 2007 году, в картине «Ирония судьбы. Продолжение».')
print(match)

Джон родился в Польше 11 сентября 1865 года , был самым младшим из 
───────────────────────────────────────────                        
семи детей в семье
None


In [103]:
import json

# parser = Parser(NAME)
seed(3)
# print(texts[1191].text)
# for text in sample(texts, 100):
#     matches = list(parser.findall(text.text))
#     if matches:
#         match = matches[0]
#         fact = match.fact
#         # show_markup(str(text), fact.spans)
#         print(json.dumps(fact.as_json, indent=2, ensure_ascii=False))

# for text in texts:
#     matches = list(parser.findall(text.text))
#     if matches:
#         match = matches[0]
#         fact = match.fact
#         # show_markup(str(text), fact.spans)
#         print(json.dumps(fact.as_json, indent=2, ensure_ascii=False))

In [122]:

match = parser.match('Джон родился 11 сентября 1865 года в Варшаве, был самым младшим из семи детей в семье')
print(match)

None


In [111]:
# parser = Parser(DATE)

print(texts[1191])
parser = Parser(ENTRY)
matches = list(parser.findall(texts[1191].text))
if matches:
        match = matches[0]
        fact = match.fact
        # show_markup(str(text), fact.spans)
        print(text), print(fact), print("\n")

seed(7)
for text in sample(texts, 100):
    matches = list(parser.findall(text.text))
    if matches:
        match = matches[0]
        fact = match.fact
        # show_markup(str(text), fact.spans)
        print(text), print(fact), print("\n")

Text(label='life', title='Шаль со спермой Джека Потрошителя и кровью его жертвы выставили на продажу', text='Шаль, которую Кэтрин Эддоус, четвертая жертва Джека Потрошителя, носила в момент убийства в 1888 году, выставили на продажу на сайте Moments in Time. По данным The New York Post, цена раритета — 4,74 миллиона долларов.Вещь продает бизнесмен и сыщик-энтузиаст Рассел Эдвардс, который приобрел платок с пятнами крови и спермы на аукционе в 2007 году. Как говорится на сайте Moments in Time, общая длина шелковой шали составляет два метра, однако некоторых фрагментов ткани недостает. Фото: momentsintime.com Этот предмет гардероба был найден одним из полицейских на месте убийства Эддоус в Лондоне. После этого шаль почти все время оставалась у потомков этого представителя Скотленд-Ярда (несколько лет она также находилась в музее лондонской полиции). Именно они выставили вещь на продажу.Приобретенный платок Эдвардс передал специалисту по молекулярной биологии, который взял пробы для теста